In [29]:
"""
ATFM
@author: Adrian Sanz Mena
"""
# pyright: reportWildcardImportFromLibrary=false

from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
from coopr.pyomo import *

# Function Definition

Given Functions

In [30]:
def S_FAP_ini(time_intervals):
    S_FAP = {}
    for t in range(time_intervals+1):  # 0..40
        S_FAP[t] = 1
    return S_FAP

def legs_ini(legs):
    legs["f1"] = (  "(STAR_1,IAF_1)","(IAF_1,FAP)" )
    legs["f2"] = (  "(STAR_1,IAF_1)","(IAF_1,FAP)")
    legs["f3"] = (  "(STAR_1,IAF_1)","(IAF_1,FAP)" )

    legs["f4"] = (  "(STAR_2,IAF_2)","(IAF_2,FAP)" )
    legs["f5"] = (  "(STAR_2,IAF_2)","(IAF_2,FAP)")
    legs["f6"] = (  "(STAR_2,IAF_2)","(IAF_2,FAP)" )

    legs["f7"] = (  "(STAR_3,IAF_3)","(IAF_3,FAP)" )
    legs["f8"] = (  "(STAR_3,IAF_3)","(IAF_3,FAP)")
    legs["f9"] = (  "(STAR_3,IAF_3)","(IAF_3,FAP)" )

    legs["f10"] = (  "(STAR_4,IAF_4)","(IAF_4,FAP)" )
    legs["f11"] = (  "(STAR_4,IAF_4)","(IAF_4,FAP)")
    legs["f12"] = (  "(STAR_4,IAF_4)","(IAF_4,FAP)" )
    
    return legs

def N_f_ini(N_f):
    N_f["f1"]  = ("STAR_1", "IAF_1", "holding_1", "FAP")
    N_f["f2"]  = ("STAR_1", "IAF_1", "holding_1", "FAP")
    N_f["f3"]  = ("STAR_1", "IAF_1", "holding_1", "FAP")

    N_f["f4"]  = ("STAR_2", "IAF_2", "holding_2", "FAP")
    N_f["f5"]  = ("STAR_2", "IAF_2", "holding_2", "FAP")
    N_f["f6"]  = ("STAR_2", "IAF_2", "holding_2", "FAP")

    N_f["f7"]  = ("STAR_3", "IAF_3", "holding_3", "FAP")
    N_f["f8"]  = ("STAR_3", "IAF_3", "holding_3", "FAP")
    N_f["f9"]  = ("STAR_3", "IAF_3", "holding_3", "FAP")

    N_f["f10"] = ("STAR_4", "IAF_4", "holding_4", "FAP")
    N_f["f11"] = ("STAR_4", "IAF_4", "holding_4", "FAP")
    N_f["f12"] = ("STAR_4", "IAF_4", "holding_4", "FAP")

    return N_f

Self-Made Functions

In [31]:
def Objective_rule(model):
    return sum(model.STAR_delay[f] + model.hold_delay[f] 
               for f in model.F)

def capacity_constraint(model, t):
    if t==0:
        return Constraint.Skip

    return sum(model.w[f, "FAP", t] - model.w[f, "FAP", t-1] 
               for f in model.F)                               <= model.S_FAP[t]

def flag_constraint(model, f, n, t):
    if t==0:
        return Constraint.Skip
    return model.w[f, n, t-1] <= model.w[f, n, t]

def star_delay(model, f):
    return model.STAR_delay[f] <= model.max_STAR_delay[f]

def hold_delay(model, f):
    return model.hold_delay[f] <= model.max_hold_delay[f]

def arrival_time(model, f):
    arrival_time = model.d_f[f] + model.STAR_delay[f] + model.hold_delay[f]
    
    for leg in model.legs[f]:
        arrival_time += model.legs_time[leg]

    return model.arrival_FAP[f] == arrival_time

def arrival_w_flag(model, f):
    return model.arrival_FAP[f] == sum(
    t * (model.w[f, model.destination_f[f], t] - model.w[f, model.destination_f[f], t-1])
    for t in model.T if t != 0
    )


# Initialize Variables and Dicts.

In [32]:
time_intervals = 50

#Sets
T = list(range(time_intervals+1))
F = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12']
K = ['STAR_1', 'STAR_2', 'STAR_3', 'STAR_4', 'FAP']

# Scheduled departure times
d_f = {
       F[0]: 1, 
       F[1]: 2, 
       F[2]: 3,
       F[3]: 1, 
       F[4]: 2, 
       F[5]: 3,
       F[6]: 1, 
       F[7]: 2, 
       F[8]: 3,
       F[9]: 1, 
       F[10]: 2, 
       F[11]: 3,
}

a_f = {
       F[0]: 1+9,
       F[1]: 2+9,
       F[2]: 3+9,
       F[3]: 1+9,
       F[4]: 2+9,
       F[5]: 3+9,
       F[6]: 1+9,
       F[7]: 2+9,
       F[8]: 3+9,
       F[9]: 1+9,
       F[10]: 2+9,
       F[11]: 3+9,
}


legs = {}

# Legs time
legs_time = {
    "(STAR_1,IAF_1)": 4,
    "(IAF_1,FAP)": 5,

    "(STAR_2,IAF_2)": 4,
    "(IAF_2,FAP)": 5,

    "(STAR_3,IAF_3)": 5,
    "(IAF_3,FAP)": 5,
    
    "(STAR_4,IAF_4)": 5,
    "(IAF_4,FAP)": 5
       }

# Origin and Destination Airport
origin_f={}
origin_f[F[0]] = K[0]
origin_f[F[1]] = K[0]
origin_f[F[2]] = K[0]
origin_f[F[3]] = K[1]
origin_f[F[4]] = K[1]
origin_f[F[5]] = K[1]
origin_f[F[6]] = K[2]
origin_f[F[7]] = K[2]
origin_f[F[8]] = K[2]
origin_f[F[9]] = K[3]
origin_f[F[10]] = K[3]
origin_f[F[11]] = K[3]

destination_f={}
destination_f[F[0]] = K[4]
destination_f[F[1]] = K[4]
destination_f[F[2]] = K[4]
destination_f[F[3]] = K[4]
destination_f[F[4]] = K[4]
destination_f[F[5]] = K[4]
destination_f[F[6]] = K[4]
destination_f[F[7]] = K[4]
destination_f[F[8]] = K[4]
destination_f[F[9]] = K[4]
destination_f[F[10]] = K[4]
destination_f[F[11]] = K[4]


# Max STAR delay for flight f
max_STAR_delay={}
for f in F:
    max_STAR_delay[f] = 3

# Max hold
max_hold_delay={}
for f in F:
    max_hold_delay[f] = 5

N_f = {}


In [33]:
############### Create a concrete model ###############
model = ConcreteModel('ATFM')

In [34]:
############### Initialize Sets and Parameters ###############
############### FEW examples are given below (complete until you get the print that you will find in the cell below) ###############

# pyright: reportAttributeAccessIssue=false 

model.T = Set(initialize=T, ordered=True)
model.F = Set(initialize=F, ordered=True)
model.K = Set(initialize=K, ordered=True)

model.legs      = Param(model.F, initialize=legs_ini(legs))
model.legs_time = Param(legs_time.keys(), initialize=legs_time)
model.S_FAP     = Param(model.T, initialize=S_FAP_ini(time_intervals))
model.N_f       = Set(model.F, initialize=N_f_ini(N_f), ordered=True)

model.d_f = Param(d_f.keys(), initialize=d_f)

model.origin_f = Param(model.F, initialize=origin_f)
model.destination_f = Param(model.F, initialize=destination_f)

model.max_STAR_delay = Param(model.F, initialize=max_STAR_delay)
model.max_hold_delay = Param(model.F, initialize=max_hold_delay)

model.STAR_delay = Var(model.F, bounds=(0, None), initialize=0)
model.hold_delay = Var(model.F, bounds=(0, None), initialize=0)

model.arrival_FAP = Var(model.F, within=Reals)


In [35]:
var = {}


# Ahora definimos la variable binaria en Pyomo solo para estos pares
model.w = Var(
    [(f,n) 
     for f in model.F 
     for n in model.N_f[f]],
    model.T,
    within=Binary,
    initialize=0
)

In [36]:
### Define Objective Function ###

# pyright: reportAttributeAccessIssue=false

model.objective_func = Objective(rule=Objective_rule, sense=minimize) 

In [37]:
# pyright: reportAttributeAccessIssue=false
# pyright: reportArgumentType=false
# pyright: reportIndexIssue=false

# Vuelos maximos en FAP <= Capcidad maxima (1)
model.capacity_constraint     = Constraint(model.T, rule=capacity_constraint)

model.T_combined = Set(dimen=3, initialize=lambda model: [
    (f, n, t) 
    for f in model.F 
    for n in model.N_f[f] 
    for t in model.T
])
# W es cumulativa
model.flag_constraint         = Constraint(model.T_combined, rule=flag_constraint) 

# Delay de F tiene que ser menor que el maximo (5)
model.star_delay_constraint   = Constraint(model.F, rule=star_delay)

# Hold de f tiene que ser menor que el maximo (3)
model.hold_delay_constraint   = Constraint(model.F, rule=hold_delay)

# Tiempo de llegada = salida + star_delay + hold + tiempos de recorrer legs
model.arrival_time_constraint = Constraint(model.F, rule=arrival_time)

# Tiempo de llegada = -1t * (w = -1)
model.link_arrival_constraint = Constraint(model.F, rule=arrival_w_flag)

In [ ]:
# Solver call
# pyright: reportAttributeAccessIssue=false

solver = SolverFactory('glpk')
#solver = SolverFactory('cplex_direct')
result = solver.solve(model)
pyomo.environ.value(model.objective_func)

objective_val = model.objective_func()
print("Objective value after minimizing:", objective_val)

In [ ]:
# Prints: one can print the complete model or just some elements of it
model.arrival_FAP.pprint()

arrival_FAP : Size=12, Index=F
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     f1 :  None :  None :  None : False :  True :  Reals
    f10 :  None :  None :  None : False :  True :  Reals
    f11 :  None :  None :  None : False :  True :  Reals
    f12 :  None :  None :  None : False :  True :  Reals
     f2 :  None :  None :  None : False :  True :  Reals
     f3 :  None :  None :  None : False :  True :  Reals
     f4 :  None :  None :  None : False :  True :  Reals
     f5 :  None :  None :  None : False :  True :  Reals
     f6 :  None :  None :  None : False :  True :  Reals
     f7 :  None :  None :  None : False :  True :  Reals
     f8 :  None :  None :  None : False :  True :  Reals
     f9 :  None :  None :  None : False :  True :  Reals


In [ ]:
model.w.pprint()

w : Size=1836, Index={('f1', 'STAR_1'), ('f1', 'IAF_1'), ('f1', 'FAP'), ('f2', 'STAR_1'), ('f2', 'IAF_1'), ('f2', 'FAP'), ('f3', 'STAR_1'), ('f3', 'IAF_1'), ('f3', 'FAP'), ('f4', 'STAR_2'), ('f4', 'IAF_2'), ('f4', 'FAP'), ('f5', 'STAR_2'), ('f5', 'IAF_2'), ('f5', 'FAP'), ('f6', 'STAR_2'), ('f6', 'IAF_2'), ('f6', 'FAP'), ('f7', 'STAR_3'), ('f7', 'IAF_3'), ('f7', 'FAP'), ('f8', 'STAR_3'), ('f8', 'IAF_3'), ('f8', 'FAP'), ('f9', 'STAR_3'), ('f9', 'IAF_3'), ('f9', 'FAP'), ('f10', 'STAR_4'), ('f10', 'IAF_4'), ('f10', 'FAP'), ('f11', 'STAR_4'), ('f11', 'IAF_4'), ('f11', 'FAP'), ('f12', 'STAR_4'), ('f12', 'IAF_4'), ('f12', 'FAP')}*T
    Key                   : Lower : Value : Upper : Fixed : Stale : Domain
         ('f1', 'FAP', 0) :     0 :     0 :     1 : False : False : Binary
         ('f1', 'FAP', 1) :     0 :     0 :     1 : False : False : Binary
         ('f1', 'FAP', 2) :     0 :     0 :     1 : False : False : Binary
         ('f1', 'FAP', 3) :     0 :     0 :     1 : False : False :